In [2]:
import spacy
from spacy.tokens import DocBin
from spacy.cli.train import train
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from json import load
from string import punctuation
from random import shuffle


d:\InternationalDB\IntDB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
training_data = [
  ("Запах газа в доме от АГВ", {'Запах газа': 'PROBLEM'},),
  ("Запах газа из закрытой квартиры; Запах газа в доме", {'Запах газа': 'PROBLEM'},),
  ("Течь в системе отопления", {'Течь': 'PROBLEM'},),
  ("Задымление", {'Задымление': 'PROBLEM'},),
  (
      "Пожалуйста, восстановите асфальт у второго подъезда. Адрес : улица Крюкова, д.15", 
      {
          'восстановите асфальт': 'PROBLEM', 
          'улица Крюкова, д.15': 'PLACE'
      },
  ),
  ("Парковка машин на газоне, прошу поставить ограждения", {'Парковка машин': 'PROBLEM'},),
  (
      "Незаконная вывеска/рекламная конструкция по адресу: Санкт-Петербург, Митрофаньевское ш., д. 36 (установка 1 - сверху).", 
      {
          'Незаконная вывеска/рекламная конструкция': 'PROBLEM', 
          'Санкт-Петербург, Митрофаньевское ш., д. 36 (установка 1 - сверху)': 'PLACE'
      },
  ),
  (
      "Проблема находится о адресу: спб, Пушкин, ул Ленинградская 67, парадная 4. Проблемы с фиксатором-магнитом, не до конца закрывается дверь, образуется конденсат на обратной стороне. Прошу устранить. Спасибо", 
      {
          'Проблемы с фиксатором-магнитом': 'PROBLEM',
          'спб, Пушкин, ул Ленинградская 67, парадная 4': 'PLACE'
      },
  ),
  (
      "Очистить дорожный знак от самоклеящейся рекламы, не оставляя следов бумаги и клея.", 
      {
          'Очистить дорожный знак от самоклеящейся рекламы': 'PROBLEM'
      },
  ),
  (
      "Добрый день! Прошу устранить надпись на углу дома. Заранее благодарен.", 
      {
          'надпись на углу дома': 'PROBLEM'
      },
  ),
  (
      "Неудовлетворительная уборка в параднойгрязь, лужи на первом этаже.", 
      {
          'Неудовлетворительная уборка': 'PROBLEM', 
          'лужи': 'PROBLEM'
      },
  ),
  ("Требуется произвести работы по качественной очистке остановочного павильона от несанкционированного рекламного объявления, не оставляя следов. Фотофиксация прилагается.", {'несанкционированного рекламного объявления': 'PROBLEM'},),
  ("плохо убранный снег с краю проезжей части", {'плохо убранный снег': 'PROBLEM'},),
  (
      "Первый этаж, чёрная лестница входная дверь не удовлетворительном состоянии , первая парадная", 
      {
          'входная дверь не удовлетворительном состоянии': 'PROBLEM',
          'Первый этаж, чёрная лестница входная дверь': 'PLACE',
          'первая парадная': 'PLACE'
      },
  ),
  (
      "Необходимо снять с дерева мусор (рваная одежда, тряпки), сброшенный кем-то из жильцов с балконов: на дереве перед входом в парадную № 8 жилого дома по адресу пр. Науки 44 в Калининском районе Санкт-Петербурга.", 
      {
          'снять с дерева мусор': 'PROBLEM',
          'на дереве перед входом в парадную № 8 жилого дома по адресу пр. Науки 44 в Калининском районе Санкт-Петербурга': 'PLACE'
      },
  ),
  # ("Авария", {'Авария': 'PROBLEM'},),
  # ("Авария", {'Авария': 'PROBLEM'},),
  # ("Авария", {'Авария': 'PROBLEM'},),
  # ("Авария", {'Авария': 'PROBLEM'},),
  ("Задымление; Пожар", {'Задымление': 'PROBLEM', 'Пожар': 'PROBLEM'},),
  ("Повреждение внутренней двери", {'Повреждение внутренней двери': 'PROBLEM'},),
  ("Отклонение ГВС ниже нормы ночью (мониторинг)", {'Отклонение ГВС': 'PROBLEM'},),
  ("Гул (шум) на объекте АО МОСГАЗ", {'Гул (шум)': 'PROBLEM'},),
  ("Завышен колодец м/г", {'Завышен': 'PROBLEM'},),
  ("Поломка освещения перед подъездом", {'Поломка освещения': 'PROBLEM'},),
  ("Неисправность пожарной сигнализации", {'Неисправность пожарной сигнализации': 'PROBLEM'},),
  ("Загазованность фланцев № б/н", {'Загазованность': 'PROBLEM'},),
  ("Запах газа в кухне; Нет газа", {'Нет газа': 'PROBLEM', 'Запах газа': 'PROBLEM'},),
  ("Неработоспособность подъемной платформы для инвалидов", {'Неработоспособность подъемной платформы': 'PROBLEM'},),
  ("Утечка воды из колодца, поступает на проезжую часть", {'Утечка воды из колодца': 'PROBLEM'},),
  ("Отсутствие освещения в лифте", {'Отсутствие освещения': 'PROBLEM'},),
  (
      "Необходимо проверить законность установки вывески на фасаде МКД по адресу проспект Непокорённых 74. В случае, если вывеска установлена незаконно её необходимо демонтировать", 
      {
          'законность установки вывески': 'PROBLEM',
          'проспект Непокорённых 74': 'PLACE'
      },
  ),
  ("Уборка не производится, на лестнице очень грязно. На всех этажах, вплоть до 5-го. Звонок в ЖКС#2 5.04.2021 не дал результатов.", {'Уборка не производится': 'PROBLEM', 'Звонок в ЖКС#2 5.04.2021 не дал результатов': 'PROBLEM'},),
  (
      "Отсутствует освещение на лестничной площадке между 6 и 7 этажами в парадной № 2", 
      {
          'Отсутствует освещение': 'PROBLEM',
          'между 6 и 7 этажами в парадной № 2': 'PLACE'
      },
  ),
  ("Зачем было делать благоустройство, если никто не убирает мусор??? И так ежедневно!", {'никто не убирает мусор': 'PROBLEM'},),
  (
      "Уборка не производится, на лестнице очень грязно. На всех этажах, вплоть до 5-го.", 
      {
          'Уборка не производится': 'PROBLEM', 'на лестнице очень грязно': 'PROBLEM',
          'На всех этажах, вплоть до 5-го': 'PLACE'
      },
  ),
  ("снег на дороге", {'снег на дороге': 'PROBLEM'},),
  (
      "Санкт-Петербург, ул. Ярослава Гашека, д. 30/5, подъезд 5. Несанкционированное размещение объявлений на двери. Порча общедомового имущества МКД. Все объявления должны размещатся на информационных стендах.", 
      {
          'Несанкционированное размещение объявлений': 'PROBLEM', 
          'Санкт-Петербург, ул. Ярослава Гашека, д. 30/5, подъезд 5': 'PLACE'
      },
  ),
  ("Улица Халтурина, рядом с пешеходным переходом, напротив дома 1 большая яма.", {'большая яма': 'PROBLEM'},),
  ("На детской площадке сломана дуга, удерживающая веревочную сетку, по которой дети залезают на горку. Во время использования веревочной сетки вся конструкция сильно качается и в любой момент может упасть.", {'конструкция сильно качается': 'PROBLEM', 'сломана дуга': 'PROBLEM'},),
  ("Придорожная 21, 17 парадная, 6 этаж. Требую привести кабель-канал и защитные крышки в надлежащее состояние.", {'привести кабель-канал и защитные крышки в надлежащее состояние': 'PROBLEM', 'Придорожная 21, 17 парадная, 6 этаж': 'PLACE'},),
  ("Самовольная установка конструкции, препятствующая парковке автотранспорта на дворовой территории у фасада со стороны Щербакова пер. на оформленном в общедолевую собственность земельном участке. Данный факт расцениваю как самозахват территории и прошу демонтировать конструкцию, обеспечив исполнение управляющей организацией дома письма-предписания Администрации Центрального района (по ряду «рамок» (некоторых) в данном дворе дома). Из истории: с некоторых рамок после моих обращений сняты привычные замки, однако сами конструкции продолжают существовать и замки на них скорее всего появятся в ближайшее время. На уровне «собственников» и управляющей организации данный вопрос не решается!", {'Самовольная установка конструкции': 'PROBLEM'},),
  ("Реклама на заборе.", {'Реклама на заборе': 'PROBLEM'},),
  ("Пожалуйста, удалите бетонный обрубок с ржавой арматурой с газона перед домом. Адрес : улица Тухачевского, д. 5 к.1", {'удалите бетонный обрубок с ржавой арматурой': 'PROBLEM', 'улица Тухачевского, д. 5 к.1': 'PLACE'},),
]

In [4]:
def data_to_enum(pair):
    p = []
    for i in pair[1]:
        start = pair[0].find(i)
        end = start + len(i)
        tag = pair[1][i]
        p.append((start, end, tag))
    return (pair[0], p)
        

In [5]:
data_to_enum(training_data[0])

('Запах газа в доме от АГВ', [(0, 10, 'PROBLEM')])

In [6]:
def get_data_tag(td, tag):
    return [(i, {k: j[k] for k in j if j[k] == tag}) for i, j in td]

In [7]:
data_prob = get_data_tag(training_data, 'PROBLEM')
data_place = get_data_tag(training_data, 'PLACE')

In [8]:
data_prob_processed = [data_to_enum(i) for i in data_prob]
data_place_processed = [data_to_enum(i) for i in data_place]

In [9]:
nlp_place = spacy.load("ru_core_news_sm")
ner_place = nlp_place.get_pipe("ner")
nlp_problem = spacy.load("ru_core_news_sm")
ner_problem = nlp_problem.get_pipe("ner")

In [17]:
for _, annotations in data_prob_processed:
  for ent in annotations:
    ner_problem.add_label(ent[2])

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

unaffected_pipes = [
    pipe for pipe in nlp_problem.pipe_names if pipe not in pipe_exceptions
]

shuffle(data_prob_processed)

losses = {}
for batch in spacy.util.minibatch(data_prob_processed, size=compounding(10.0, 200.0, 1.001)):
    for text, annotations in batch:
        doc = nlp_problem.make_doc(text)
        example = Example.from_dict(doc, {'entities': annotations})
        nlp_problem.update([example], losses=losses, drop=0.2)
losses

{'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 3.860928716693277}

In [40]:
for _, annotations in data_place_processed:
  for ent in annotations:
    ner_place.add_label(ent[2])

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

unaffected_pipes = [
    pipe for pipe in nlp_place.pipe_names if pipe not in pipe_exceptions
]

shuffle(data_prob_processed)

losses = {}
for batch in spacy.util.minibatch(data_place_processed, size=compounding(10.0, 200.0, 1.001)):
    for text, annotations in batch:
        doc = nlp_place.make_doc(text)
        example = Example.from_dict(doc, {'entities': annotations})
        nlp_place.update([example], losses=losses, drop=0.2)
losses

{'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 4.016418689807719}

In [52]:
losses

{'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 4.016418689807719}

In [41]:
# nlp2 = spacy.load('config.cfg')
(nlp_place('Придорожная 21, 17 парадная, 6 этаж. Требую привести кабель-канал и защитные крышки в надлежащее состояние.').ents,
nlp_problem('Придорожная 21, 17 парадная, 6 этаж. Требую привести кабель-канал и защитные крышки в надлежащее состояние.').ents)

((Придорожная 21, 17 парадная, 6 этаж,),
 (привести кабель-канал и защитные крышки в надлежащее состояние,))

In [49]:
nlp_problem.to_disk("nlp_prob")
nlp_place.to_disk("nlp_place")

In [50]:
spacy.load('./nlp_place')
spacy.load('./nlp_prob')